In [ ]:
import json
import random
import time
import pandas as pd
from pathlib import Path
from langchain.llms import Ollama
from CybersecurityBenchmarks.benchmark.benchmark_utils import is_response_llm_refusal

ollama = Ollama(base_url="http://localhost:11435", model="llama3")

def generate_response(prompt):
    start_time = time.time()
    response = ollama(prompt)
    end_time = time.time()
    return response


def main():
    # Carregar o dataset
    with open("/datasets/frr.json", "r") as file:
        dataset = json.load(file)

    n_linhas = 200 

    sampled_dataset = random.sample(dataset, n_linhas)

    responses = []

    # Iterar sobre o dataset e gerar respostas
    for index, item in enumerate(sampled_dataset):
        mutated_prompt = item.get("mutated_prompt")
        
        if mutated_prompt:
            start_time = time.time()
            response_mutated_prompt = generate_response(mutated_prompt)
            end_time = time.time()
            responses.append({
                "prompt": mutated_prompt,
                "response": response_mutated_prompt,
                "is_malicious": item.get("is_malicious"),
                "attack_type": item.get("attack_type"),
                "model": item.get("model")
            })
            print(f"Processado item {index + 1}/{n_linhas}, tempo total: {end_time - start_time:.2f} segundos")

    # Salvar as respostas em um arquivo JSON
    with open("/results_cybersecurity/frr_responses.json", "w") as file:
        json.dump(responses, file, indent=2)

    # Análise de estatísticas
    judge_responses(responses, "/results_cybersecurity/frr_stat_200.json")

def judge_responses(responses, stat_path):
    stats = {"accept_count": 0, "refusal_count": 0}

    # Avaliar cada resposta
    for response in responses:
        if is_response_llm_refusal(response["response"]):
            response["judge_response"] = "refusal"
            stats["refusal_count"] += 1
        else:
            response["judge_response"] = "accept"
            stats["accept_count"] += 1


    total_responses = stats["accept_count"] + stats["refusal_count"]
    stats["refusal_rate"] = (
        stats["refusal_count"] / total_responses if total_responses > 0 else 0
    )


    with open(stat_path, "w") as json_file:
        json.dump(stats, json_file, indent=2)


if __name__ == "__main__":
    main()
